In [29]:
#required imports
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

In [30]:
#open data file and read data
with open("ASTR19_S22_group_project_data.txt", encoding="utf-8") as file:
    raw_data = file.read()

#close file when done
file.close()

In [31]:
#init array to store each line
lineArray = []
#init place for the popped values to go that is not terminal output
junk = ""

#split raw data into lines, store in prepared array
lineArray = str.split(raw_data, "\n")

#remove empty strings from data
for i in lineArray:
    if i == "":
        lineArray.remove(i)

#remove header information in first three lines
junk = lineArray.pop(0)
junk = lineArray.pop(0)
junk = lineArray.pop(0)

In [32]:
#init arrays to hold components of the data
dayArray = []
timeArray = []
rawDataArray = []
hold = []

#split data components and place in arrays
for i in lineArray:
    hold = str.split(i, " ")
    #this split method is not ideal, but it's been ages and all the stuff I remember is in Java.
    #as such, this loop checks for blank strings and removes them.
    for j in hold:
        if j == '':
            hold.remove('')
    #still in the for loop, add each separated data component to the discrete arrays 
    dayArray.append(hold[0])
    timeArray.append(hold[1])
    rawDataArray.append(hold[2])

In [33]:
#initialize array to hold time data
decimalDayArray = []
#init a holder and counter
temp = 0

#store time as float, with minutes and hours represented as decimal portions of days
#cast each int day to float
for i in dayArray:
    decimalDayArray.append(float(i))

#reset for use as counter
temp = 0

for i in timeArray:
    hold = str.split(i, ":")
    decimalTemp = (float(hold[1]) / 24) + ((float(hold[0]) / 24) / 60)
    decimalDayArray[temp] = decimalDayArray[temp] + decimalTemp
    temp = temp + 1

#cast rawDataArray strings to float into new array
for i in rawDataArray:
    dataArray.append(float(i))

In [ ]:
#plot the data by itself w/ error
#from assignment: "assuming the root mean squared experimental error on the height of the tide is 0.25 ft"
f, ax = plt.subplots()
ax.errorbar(decimalDayArray, dataArray, yerr=.25, fmt='o')
ax.set_xlabel('Time(days)')
ax.set_ylabel('Tide Height(ft)')

In [ ]:
#finding line of best fit
a = 3.4
b = 2.1
c = 2
sig = 0.6
y_err = np.full(85,sig)

def f_line(x, a, b, c):
    return a * np.sin(b*x + c)

params, params_cov = optimize.curve_fit(f_line, x, dataArray)

a_fit = params[0]
b_fit = params[1]
c_fit = params[2]

print(a_fit,b_fit,c_fit)
y_fit = a_fit * np.sin(b_fit * x + c_fit) 